In [ ]:
!apt-get update
!apt-get install graphviz libgraphviz-dev -y
!pip install pygraphviz

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,148 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,486 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,439 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,223 kB]
Hit:13 https://ppa.launc

In [ ]:
import numpy as np
import networkx as nx

def hits_algorithm_nx(graph, max_iter=100, tol=1e-6):

    nodes = list(graph.nodes)
    hubs = {node: 1.0 for node in nodes}
    authorities = {node: 1.0 for node in nodes}

    for _ in range(max_iter):
        # Update authority scores
        new_authorities = {node: 0.0 for node in nodes}
        for node in nodes:
            for neighbor in graph.successors(node):  # Sum hub scores of incoming links
                new_authorities[neighbor] += hubs[node]

        # Update hub scores
        new_hubs = {node: 0.0 for node in nodes}
        for node in nodes:
            for neighbor in graph.successors(node):  # Sum authority scores of outgoing links
                new_hubs[node] += authorities[neighbor]

        # Max normalization
        max_authority = max(new_authorities.values())
        max_hub = max(new_hubs.values())

        if max_authority > 0:
            new_authorities = {node: score / max_authority for node, score in new_authorities.items()}
        if max_hub > 0:
            new_hubs = {node: score / max_hub for node, score in new_hubs.items()}

        # Convergence check
        authority_diff = sum(abs(new_authorities[node] - authorities[node]) for node in nodes)
        hub_diff = sum(abs(new_hubs[node] - hubs[node]) for node in nodes)
        if authority_diff < tol and hub_diff < tol:
            break

        authorities = new_authorities
        hubs = new_hubs

    return hubs, authorities

if __name__ == "__main__":
    # Create a directed graph
    G = nx.DiGraph()
    G.add_edges_from([
        ("A", "B"), ("A", "C"),
        ("B", "C"),
        ("C", "A"),
        ("E", "E")  # Self-loop
    ])

    # Run custom HITS algorithm
    custom_hubs, custom_authorities = hits_algorithm_nx(G)

    # Run NetworkX's built-in HITS algorithm
    nx_hubs, nx_authorities = nx.hits(G, max_iter=100, tol=1e-6)

    # Display results
    print("Custom Hub Scores:")
    for node, score in custom_hubs.items():
        print(f"  {node}: {score:.4f}")

    print("\nCustom Authority Scores:")
    for node, score in custom_authorities.items():
        print(f"  {node}: {score:.4f}")

    print("\nNetworkX Hub Scores:")
    for node, score in nx_hubs.items():
        print(f"  {node}: {score:.4f}")

    print("\nNetworkX Authority Scores:")
    for node, score in nx_authorities.items():
        print(f"  {node}: {score:.4f}")

    # Verify if both are close
    hub_close = all(np.isclose(custom_hubs[node], nx_hubs[node], atol=1e-4) for node in G.nodes)
    authority_close = all(np.isclose(custom_authorities[node], nx_authorities[node], atol=1e-4) for node in G.nodes)

    print("\nVerification:")
    print(f"Hubs close: {hub_close}")
    print(f"Authorities close: {authority_close}")


Custom Hub Scores:
  A: 1.0000
  B: 0.6180
  C: 0.0000
  E: 0.0000

Custom Authority Scores:
  A: 0.0000
  B: 0.6180
  C: 1.0000
  E: 0.0000

NetworkX Hub Scores:
  A: 0.6180
  B: 0.3820
  C: 0.0000
  E: 0.0000

NetworkX Authority Scores:
  A: 0.0000
  B: 0.3820
  C: 0.6180
  E: 0.0000

Verification:
Hubs close: False
Authorities close: False


In [1]:
import numpy as np
import networkx as nx

def hits_algorithm_nx_l1(graph, max_iter=100, tol=1e-6):
    nodes = list(graph.nodes)
    hubs = {node: 1.0 for node in nodes}
    authorities = {node: 1.0 for node in nodes}

    for _ in range(max_iter):
        # Update authority scores
        new_authorities = {node: 0.0 for node in nodes}
        for node in nodes:
            for neighbor in graph.successors(node):  # Sum hub scores of incoming links
                new_authorities[neighbor] += hubs[node]

        # Update hub scores
        new_hubs = {node: 0.0 for node in nodes}
        for node in nodes:
            for neighbor in graph.successors(node):  # Sum authority scores of outgoing links
                new_hubs[node] += authorities[neighbor]

        # L1 normalization
        sum_authority = sum(new_authorities.values())
        if sum_authority > 0:
            new_authorities = {node: score / sum_authority for node, score in new_authorities.items()}

        sum_hub = sum(new_hubs.values())
        if sum_hub > 0:
            new_hubs = {node: score / sum_hub for node, score in new_hubs.items()}

        # Convergence check
        authority_diff = sum(abs(new_authorities[node] - authorities[node]) for node in nodes)
        hub_diff = sum(abs(new_hubs[node] - hubs[node]) for node in nodes)
        if authority_diff + hub_diff < tol:
            break

        authorities = new_authorities
        hubs = new_hubs

    return hubs, authorities

if __name__ == "__main__":
    # Create a directed graph
    G = nx.DiGraph()
    G.add_edges_from([
        ("A", "B"), ("A", "C"),
        ("B", "C"),
        ("C", "A"),
        ("E", "E")  # Self-loop
    ])

    # Run custom HITS algorithm with L1 norm
    custom_hubs, custom_authorities = hits_algorithm_nx_l1(G)

    # Run NetworkX's built-in HITS algorithm
    nx_hubs, nx_authorities = nx.hits(G, max_iter=100, tol=1e-6)

    # Display results
    print("Custom Hub Scores:")
    for node, score in custom_hubs.items():
        print(f"  {node}: {score:.4f}")

    print("\nCustom Authority Scores:")
    for node, score in custom_authorities.items():
        print(f"  {node}: {score:.4f}")

    print("\nNetworkX Hub Scores:")
    for node, score in nx_hubs.items():
        print(f"  {node}: {score:.4f}")

    print("\nNetworkX Authority Scores:")
    for node, score in nx_authorities.items():
        print(f"  {node}: {score:.4f}")

    # Verify if both are close
    hub_close = all(np.isclose(custom_hubs[node], nx_hubs[node], atol=1e-4) for node in G.nodes)
    authority_close = all(np.isclose(custom_authorities[node], nx_authorities[node], atol=1e-4) for node in G.nodes)

    print("\nVerification:")
    print(f"Hubs close: {hub_close}")
    print(f"Authorities close: {authority_close}")


Custom Hub Scores:
  A: 0.6180
  B: 0.3820
  C: 0.0000
  E: 0.0000

Custom Authority Scores:
  A: 0.0000
  B: 0.3820
  C: 0.6180
  E: 0.0000

NetworkX Hub Scores:
  A: 0.6180
  B: 0.3820
  C: 0.0000
  E: 0.0000

NetworkX Authority Scores:
  A: 0.0000
  B: 0.3820
  C: 0.6180
  E: 0.0000

Verification:
Hubs close: True
Authorities close: True


In [ ]:
def hits_with_numpy(adjacency_matrix, max_iter=100, tol=1e-6):
    """
    Implements the HITS algorithm using NumPy.

    Parameters:
        adjacency_matrix (np.ndarray): The adjacency matrix of the directed graph.
                                       adjacency_matrix[i][j] = 1 if there is an edge from node i to node j.
        max_iter (int): Maximum number of iterations.
        tol (float): Convergence tolerance.

    Returns:
        (np.ndarray, np.ndarray): Authority and hub scores for each node.
    """
    # Initialize hub scores
    num_nodes = adjacency_matrix.shape[0]
    hubs = np.ones(num_nodes)
    authorities = np.ones(num_nodes)

    for _ in range(max_iter):
        # Update authority scores: a = A.T @ h
        new_authorities = adjacency_matrix.T @ hubs

        # Update hub scores: h = A @ a
        new_hubs = adjacency_matrix @ new_authorities

        # Normalize scores
        new_authorities /= np.linalg.norm(new_authorities, ord=2)
        new_hubs /= np.linalg.norm(new_hubs, ord=2)

        # Check for convergence
        if np.allclose(new_authorities, authorities, atol=tol) and np.allclose(new_hubs, hubs, atol=tol):
            break

        authorities = new_authorities
        hubs = new_hubs

    return authorities, hubs


In [ ]:
import networkx as nx
import numpy as np
import graphviz
# from networkx.drawing.nx_agraph import from_agraph  #No need for this import
from pygraphviz import AGraph  #Import AGraph for proper conversion

def graphviz_to_adjacency_matrix_with_networkx(graphviz_graph):
    """
    Converts a Graphviz graph to an adjacency matrix using NetworkX.

    Parameters:
        graphviz_graph (graphviz.Digraph or graphviz.Graph): Input Graphviz graph.

    Returns:
        np.ndarray: Adjacency matrix of the graph.
        list: List of nodes in the order corresponding to the matrix.
    """
    # Convert Graphviz graph to NetworkX graph
    # Convert the graphviz graph to a string representation
    dot_string = graphviz_graph.source

    # Create an AGraph object from the string
    agraph_obj = AGraph(string=dot_string)

    # Use from_agraph to convert AGraph to NetworkX graph
    nx_graph = nx.DiGraph(nx.nx_agraph.from_agraph(agraph_obj))

    # Get adjacency matrix
    adjacency_matrix = nx.to_numpy_array(nx_graph, dtype=int)

    # Get list of nodes
    nodes = list(nx_graph.nodes())

    return adjacency_matrix, nodes

# Example usage
if __name__ == "__main__":


    # Define a Graphviz graph
    dot = graphviz.Digraph()
    dot.edge('A', 'B')
    dot.edge('A', 'C')
    dot.edge('B', 'C')
    dot.edge('B', 'D')
    dot.edge('C', 'A')
    dot.edge('C', 'D')
    dot.edge('D', 'A')

    # Get adjacency matrix
    adjacency_matrix, nodes = graphviz_to_adjacency_matrix_with_networkx(dot)

    print("Adjacency Matrix:")
    print(adjacency_matrix)
    print("Nodes:", nodes)

    authorities, hubs = hits_with_numpy(adjacency_matrix)

    # Print results
    print("Authority Scores:", authorities)
    print("Hub Scores:", hubs)

Adjacency Matrix:
[[0 1 1 0]
 [0 0 1 1]
 [1 0 0 1]
 [1 0 0 0]]
Nodes: ['A', 'B', 'C', 'D']
Authority Scores: [0.42853966 0.2280039  0.57733744 0.65654357]
Hub Scores: [0.42851318 0.65653437 0.57736073 0.22802119]
